In [196]:
import pandas as pd 
import numpy as np
import requests
from bs4 import BeautifulSoup as soup 
import altair as alt 
import plotly.express as px
from IPython.display import clear_output

In [197]:
dirty = pd.read_csv('datasets/gpa_filtered.csv')
dirty

,Unnamed: 0.1,Unnamed: 0,Year,Term,YearTerm,Subject,Number,Course Title,Sched Type,A+,...,D+,D,D-,F,W,Students,Primary Instructor,Total_Grade_Points,Total_Students,Overall_GPA
0,0,10,2025,Winter,2025-wi,ECON,102,Microeconomic Principles,ONL,44,...,0,1,0,0,0,97,"DiIanni, Isaac",371.01,97,3.824845
1,1,14,2025,Winter,2025-wi,IS,445,Data Visualization,ONL,18,...,0,0,0,0,1,31,"Besser, Stephanie",122.67,31,3.957097
2,2,826,2024,Fall,2024-fa,CS,124,Intro to Computer Science I,OLC,0,...,0,16,0,34,16,1107,"Challen, Geoffrey",4033.21,1107,3.643369
3,3,1048,2024,Fall,2024-fa,ECON,102,Microeconomic Principles,DIS,15,...,0,0,0,0,0,48,"Cueva Chavez, Ronald",163.69,48,3.410208
4,4,1049,2024,Fall,2024-fa,ECON,102,Microeconomic Principles,DIS,7,...,0,0,0,0,0,48,"Cueva Chavez, Ronald",162.35,48,3.382292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1145,1145,73625,2010,Spring,2010-sp,MATH,231,Calculus II,LCD,2,...,0,3,2,0,0,24,"Szuta, Patryk",67.69,24,2.820417
1146,1146,73626,2010,Spring,2010-sp,MATH,231,Calculus II,LCD,0,...,0,0,0,1,1,22,"Solie, Brent",63.67,22,2.894091
1147,1147,74488,2010,Summer,2010-su,ECON,102,Microeconomic Principles,LCD,0,...,1,0,0,0,0,33,"Vazquez, Jose",103.70,33,3.142424
1148,1148,74489,2010,Summer,2010-su,ECON,102,Microeconomic Principles,LCD,1,...,0,1,0,0,0,26,"Arvin-Rad, Hassan",79.02,26,3.039231


In [198]:
dirty = dirty.drop('Unnamed: 0', axis=1)
dirty

,Unnamed: 0.1,Year,Term,YearTerm,Subject,Number,Course Title,Sched Type,A+,A,...,D+,D,D-,F,W,Students,Primary Instructor,Total_Grade_Points,Total_Students,Overall_GPA
0,0,2025,Winter,2025-wi,ECON,102,Microeconomic Principles,ONL,44,29,...,0,1,0,0,0,97,"DiIanni, Isaac",371.01,97,3.824845
1,1,2025,Winter,2025-wi,IS,445,Data Visualization,ONL,18,11,...,0,0,0,0,1,31,"Besser, Stephanie",122.67,31,3.957097
2,2,2024,Fall,2024-fa,CS,124,Intro to Computer Science I,OLC,0,819,...,0,16,0,34,16,1107,"Challen, Geoffrey",4033.21,1107,3.643369
3,3,2024,Fall,2024-fa,ECON,102,Microeconomic Principles,DIS,15,10,...,0,0,0,0,0,48,"Cueva Chavez, Ronald",163.69,48,3.410208
4,4,2024,Fall,2024-fa,ECON,102,Microeconomic Principles,DIS,7,13,...,0,0,0,0,0,48,"Cueva Chavez, Ronald",162.35,48,3.382292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1145,1145,2010,Spring,2010-sp,MATH,231,Calculus II,LCD,2,4,...,0,3,2,0,0,24,"Szuta, Patryk",67.69,24,2.820417
1146,1146,2010,Spring,2010-sp,MATH,231,Calculus II,LCD,0,3,...,0,0,0,1,1,22,"Solie, Brent",63.67,22,2.894091
1147,1147,2010,Summer,2010-su,ECON,102,Microeconomic Principles,LCD,0,8,...,1,0,0,0,0,33,"Vazquez, Jose",103.70,33,3.142424
1148,1148,2010,Summer,2010-su,ECON,102,Microeconomic Principles,LCD,1,4,...,0,1,0,0,0,26,"Arvin-Rad, Hassan",79.02,26,3.039231


In [199]:
grouped = (dirty.groupby(['Primary Instructor', 'Course Title'], as_index=False).agg(
        simple_gpa=('Overall_GPA', 'mean'),
        total_grade_points=('Total_Grade_Points', 'sum'),
        total_students=('Total_Students', 'sum')
    )
)
grouped['weighted_gpa'] = grouped['total_grade_points'] / grouped['total_students']
grouped = grouped.rename(columns={'Primary Instructor' : 'primary_instructor', 'Course Title' : 'course_title'})
grouped

,primary_instructor,course_title,simple_gpa,total_grade_points,total_students,weighted_gpa
0,"Agirdas, Cagdas",Microeconomic Principles,3.101918,434.09,140,3.100643
1,"Ahlgren, Scott",Calculus II,2.887670,3800.26,1369,2.775939
2,"Ahmed, Iftikhar",Calculus II,2.667727,58.69,22,2.667727
3,"Albin, Pierre",Calculus II,2.857759,828.75,290,2.857759
4,"Ambardar, Shivangi",Microeconomic Principles,3.239827,1283.75,396,3.241793
...,...,...,...,...,...,...
195,"Yu, Yueyuan",Microeconomic Principles,3.026555,746.64,241,3.098091
196,"Zhang, Cong",Microeconomic Principles,3.243892,1402.52,432,3.246574
197,"Zharnitsky, Vadim",Calculus II,2.717079,1906.83,700,2.724043
198,"Zubareva, Daria",Microeconomic Principles,2.942513,226.70,77,2.944156


In [200]:
def last_first_to_first_last(name):
    parts = [p.strip() for p in name.split(',')]
    if len(parts) == 2:
        last, first = parts
        return f"{first} {last}"
    return name

grouped['primary_instructor'] = grouped['primary_instructor'].apply(last_first_to_first_last)
grouped

,primary_instructor,course_title,simple_gpa,total_grade_points,total_students,weighted_gpa
0,Cagdas Agirdas,Microeconomic Principles,3.101918,434.09,140,3.100643
1,Scott Ahlgren,Calculus II,2.887670,3800.26,1369,2.775939
2,Iftikhar Ahmed,Calculus II,2.667727,58.69,22,2.667727
3,Pierre Albin,Calculus II,2.857759,828.75,290,2.857759
4,Shivangi Ambardar,Microeconomic Principles,3.239827,1283.75,396,3.241793
...,...,...,...,...,...,...
195,Yueyuan Yu,Microeconomic Principles,3.026555,746.64,241,3.098091
196,Cong Zhang,Microeconomic Principles,3.243892,1402.52,432,3.246574
197,Vadim Zharnitsky,Calculus II,2.717079,1906.83,700,2.724043
198,Daria Zubareva,Microeconomic Principles,2.942513,226.70,77,2.944156


In [201]:
grouped[
    grouped['primary_instructor']
           .str.contains('Daniel', case=False, na=False)
]

,primary_instructor,course_title,simple_gpa,total_grade_points,total_students,weighted_gpa
19,Daniel Berwick Evans,Calculus II,2.964776,2843.57,959,2.965141
74,Daniel Halim,Microeconomic Principles,3.411210,535.65,157,3.411783


In [202]:
grouped['primary_instructor'].unique()

array(['Cagdas Agirdas', 'Scott Ahlgren', 'Iftikhar Ahmed',
       'Pierre Albin', 'Shivangi Ambardar', 'Sadra Amiri Moghadam',
       'Mateo Arbelaez Parra', 'Hassan Arvin-Rad', 'Deniz Ay',
       'Diogo Baerlocher Carv', 'Manijeh Bahreini Esfaha',
       'Diloa Bailey Athias', 'Di Bao', 'Yuliy Baryshnikov',
       'Riddhasattam Basu', 'Anahid Bauer', 'Jesse Beder',
       'Sebastiao Benete Reis De', 'Maarten Bergvelt',
       'Daniel Berwick Evans', 'Stephanie Besser', 'Daria Bottan',
       'Steven Bradlow', 'Michael Brannan', 'Jared Bronski',
       'David Buller', 'Justin Burner', 'Maria Cantet', 'Bruce Carpenter',
       'Jiyoung Chae', 'Geoffrey Challen', 'Eui Soon Chang',
       'Youning Chen', 'Yuci Chen', 'Zhuo Chen', 'Tzu-Chang Cheng',
       'Hee Pyung Cho', 'Jer-Chin Chuang', 'Evangelos Constantinou',
       'Diana Cooke', 'Adriana Corredor Waldro', 'Ronald Cueva Chavez',
       "John D'Angelo", 'Sergio De Holanda Roch', 'Robert DeVille',
       'Ethan Dee', 'Keith Dent', 

In [203]:
profs = pd.read_csv('datasets/full_dataset.csv')
profs

,Course Number,CRN,Type,Section,Time,Days,Location,Instructor
0,CS 124,74477,Lecture,AL1,ARRANGED,-,-,"Challen, G"
1,CS 124,75242,Lecture,AL2,ARRANGED,-,-,"Challen, G"
2,CS 124,74481,Lab/Discussion,QBA,10:00 AM - 10:50 AM,T,4025 - Campus Instructional Facility,"Challen, G"
3,CS 124,74482,Lab/Discussion,QBB,10:00 AM - 10:50 AM,T,4029 - Campus Instructional Facility,"Challen, G"
4,CS 124,74490,Lab/Discussion,QBC,11:00 AM - 11:50 AM,T,4025 - Campus Instructional Facility,"Challen, G"
...,...,...,...,...,...,...,...,...
190,STAT 107,71689,Laboratory-Discussion,Y40,03:30 PM - 04:50 PM,F,313 - Davenport Hall,NaN
191,STAT 107,78212,Laboratory-Discussion,Y41,03:30 PM - 04:50 PM,F,"1136 - Literatures, Cultures, & Ling",NaN
192,STAT 107,78213,Laboratory-Discussion,Y42,05:00 PM - 06:20 PM,F,313 - Davenport Hall,NaN
193,STAT 107,78771,Laboratory-Discussion,Y43,05:00 PM - 06:20 PM,F,"G32 - Literatures, Cultures, & Ling",NaN


In [204]:
profs['Instructor'].unique()

array(['Challen, G', 'DiIanni, I', 'Buckley, B', 'Turk, M', 'Naiman, J',
       'Bahreini Esfahani, M', 'Hoffmeister, A', 'Bronski, J', nan,
       'Chuang, J', 'Berwick Evans, D',
       'Fagen-Ulmschneider, W; Flanagan, K'], dtype=object)

In [205]:
len(profs['Instructor'].unique())

12

In [206]:
rmp_data = {
    'primary_instructor' : ['Geoffrey Challen', 'Isaac DiIanni', 'Bryan Buckley', 
                    'Manijeh Bahreini Esfaha', 'April Hoffmeister', 'Jared Bronski', 
                    'Jer-Chin Chuang', 'Daniel Berwick Evans', 'Wade Fagen-Ulmschneider',
                    'Karle Flanagan'], 
    'rating' : [3.3, 4.4, 3.8, 3.9, 4.9, 4.5, 2.3, 4.1, 4.6, 4.9], 
    'level_of_difficulty' : [3.7, 2.6, 2.9, 2.9, 2.9, 2.8, 4.3, 3.6, 2.6, 1.9], 
    'would_take_again' : [0.6, 0.83, 0.67, 0.78, 0.99, 1.0, 0.3, 0.8, 0.92, 0.97], 
    'total_ratings' : [173, 156, 94, 49, 65, 79, 68, 15, 64, 252]}
rmp_df = pd.DataFrame(rmp_data)
rmp_df

,primary_instructor,rating,level_of_difficulty,would_take_again,total_ratings
0,Geoffrey Challen,3.3,3.7,0.60,173
1,Isaac DiIanni,4.4,2.6,0.83,156
2,Bryan Buckley,3.8,2.9,0.67,94
3,Manijeh Bahreini Esfaha,3.9,2.9,0.78,49
4,April Hoffmeister,4.9,2.9,0.99,65
5,Jared Bronski,4.5,2.8,1.00,79
6,Jer-Chin Chuang,2.3,4.3,0.30,68
7,Daniel Berwick Evans,4.1,3.6,0.80,15
8,Wade Fagen-Ulmschneider,4.6,2.6,0.92,64
9,Karle Flanagan,4.9,1.9,0.97,252


In [207]:
df = rmp_df.merge(grouped, on='primary_instructor', how='left')
df

,primary_instructor,rating,level_of_difficulty,would_take_again,total_ratings,course_title,simple_gpa,total_grade_points,total_students,weighted_gpa
0,Geoffrey Challen,3.3,3.7,0.60,173,Intro to Computer Science I,3.564427,23865.20,6608.0,3.611562
1,Isaac DiIanni,4.4,2.6,0.83,156,Microeconomic Principles,3.216907,15924.64,4623.0,3.444655
2,Bryan Buckley,3.8,2.9,0.67,94,NaN,NaN,NaN,NaN,NaN
3,Manijeh Bahreini Esfaha,3.9,2.9,0.78,49,Calculus II,3.095914,11044.51,3556.0,3.105880
4,April Hoffmeister,4.9,2.9,0.99,65,NaN,NaN,NaN,NaN,NaN
5,Jared Bronski,4.5,2.8,1.00,79,Calculus II,2.994208,9527.32,3190.0,2.986621
6,Jer-Chin Chuang,2.3,4.3,0.30,68,Linear Algebra w Computat Appl,3.282757,15058.10,4579.0,3.288513
7,Daniel Berwick Evans,4.1,3.6,0.80,15,Calculus II,2.964776,2843.57,959.0,2.965141
8,Wade Fagen-Ulmschneider,4.6,2.6,0.92,64,NaN,NaN,NaN,NaN,NaN
9,Karle Flanagan,4.9,1.9,0.97,252,Data Science Discovery,3.571818,15598.41,4345.0,3.589968


In [208]:
df['course_code'] = ['CS124', 'ECON102', np.nan, 'MATH231', np.nan, 'MATH231','MATH257', 'MATH231', np.nan, 'STAT107']
df

,primary_instructor,rating,level_of_difficulty,would_take_again,total_ratings,course_title,simple_gpa,total_grade_points,total_students,weighted_gpa,course_code
0,Geoffrey Challen,3.3,3.7,0.60,173,Intro to Computer Science I,3.564427,23865.20,6608.0,3.611562,CS124
1,Isaac DiIanni,4.4,2.6,0.83,156,Microeconomic Principles,3.216907,15924.64,4623.0,3.444655,ECON102
2,Bryan Buckley,3.8,2.9,0.67,94,NaN,NaN,NaN,NaN,NaN,NaN
3,Manijeh Bahreini Esfaha,3.9,2.9,0.78,49,Calculus II,3.095914,11044.51,3556.0,3.105880,MATH231
4,April Hoffmeister,4.9,2.9,0.99,65,NaN,NaN,NaN,NaN,NaN,NaN
5,Jared Bronski,4.5,2.8,1.00,79,Calculus II,2.994208,9527.32,3190.0,2.986621,MATH231
6,Jer-Chin Chuang,2.3,4.3,0.30,68,Linear Algebra w Computat Appl,3.282757,15058.10,4579.0,3.288513,MATH257
7,Daniel Berwick Evans,4.1,3.6,0.80,15,Calculus II,2.964776,2843.57,959.0,2.965141,MATH231
8,Wade Fagen-Ulmschneider,4.6,2.6,0.92,64,NaN,NaN,NaN,NaN,NaN,NaN
9,Karle Flanagan,4.9,1.9,0.97,252,Data Science Discovery,3.571818,15598.41,4345.0,3.589968,STAT107


In [209]:
# df.to_csv('datasets/fa25_profs_gpa_rmp.csv')

In [219]:
# 1. Load & prepare your master dataset
df_master = pd.read_csv('datasets/fa25_profs_gpa_rmp.csv')

# 2. Coerce key columns to numeric and drop any incomplete rows
for col in ['weighted_gpa', 'rating', 'level_of_difficulty']:
    df_master[col] = pd.to_numeric(df_master[col], errors='coerce')
df_master = df_master.dropna(subset=['weighted_gpa','rating','level_of_difficulty','course_code'])

# 3. State: user’s current selection & what we’ll plot
selected_codes = []
plot_df = pd.DataFrame(columns=df_master.columns)

min_sz, max_sz = 10, 50
df_master['marker_size'] = (
    (df_master['level_of_difficulty'] - 1)
      / (5 - 1)
      * (max_sz - min_sz)
      + min_sz
)

# (Optionally you can do the same for rating if you want size by rating
#  df_master['marker_size'] = ((df_master['rating'] - 1)/(5-1))*(max_sz-min_sz)+min_sz
# )

# 2) In your chart factory, swap size → 'marker_size'  
def make_chart(dataframe):
    fig = px.scatter(
        dataframe,
        x='course_code',
        y='weighted_gpa',
        color='rating',
        size='marker_size',        # use our precomputed pixel sizes
        size_max=max_sz,           # just to enforce the ceiling
        color_continuous_scale='Viridis',
        labels={
            'course_code':'Course Code',
            'weighted_gpa':'Weighted GPA',
            'rating':'Prof Rating',
            'marker_size':'Difficulty (scaled)'
        },
        title='Course GPA vs Prof Rating (size = Difficulty)'
    )
    fig.update_traces(
        marker=dict(line=dict(width=1, color='white'), opacity=0.85)
    )
    fig.update_layout(template='plotly_white', width=700, height=450)
    return fig

# 5. Display helper: clears and redraws
def update_chart():
    clear_output(wait=True)
    if plot_df.empty:
        print("No courses selected")
    else:
        make_chart(plot_df).show()

# 6. Add a course (by its code) to the plot
def add_course(code: str):
    global plot_df, selected_codes
    code = code.strip().upper()
    if code in selected_codes:
        print(f"⚠️  {code} is already selected.")
        return
    subset = df_master[df_master['course_code'] == code]
    if subset.empty:
        print(f"❌  No data found for course code: {code}")
        return
    selected_codes.append(code)
    plot_df = pd.concat([plot_df, subset], ignore_index=True)
    update_chart()

# 7. Remove a course (by its code) from the plot
def remove_course(code: str):
    global plot_df, selected_codes
    code = code.strip().upper()
    if code not in selected_codes:
        print(f"⚠️  {code} is not in the current selection.")
        return
    selected_codes.remove(code)
    plot_df = plot_df[plot_df['course_code'] != code]
    update_chart()

# 8. Kick off the demo
print("🎓  Course–Professor GPA Interactive Demo")
update_chart()

No courses selected


In [220]:
add_course('CS124')
add_course('MATH257')
add_course('MATH231')
add_course('ECON102')
add_course('STAT107')

In [221]:
remove_course('CS124')

In [291]:
import altair as alt

def make_grouped_bar_chart(df, chart_width=700, chart_height=450):
    """
    Returns an Altair grouped‐bar chart where:
      • x = course_code
      • bars within each course are one per primary_instructor (side-by-side)
      • y = weighted_gpa
      • color = primary_instructor
    """
    return (
        alt.Chart(df)
           # thicker bars
           .mark_bar(size=40)
           .encode(
               x=alt.X(
                   'course_code:O',
                   title='Course',
                   axis=alt.Axis(labelAngle=-45, labelFontSize=12, titleFontSize=14),
                   scale=alt.Scale(
                       paddingInner=0.352,  # space between professors in the same course
                       padding=0.1        # overall padding on the ends
                   )
               ),
               # side-by-side grouping by instructor
               xOffset='primary_instructor:N',
               y=alt.Y(
                   'weighted_gpa:Q',
                   title='Weighted GPA',
                   axis=alt.Axis(labelFontSize=12, titleFontSize=14)
               ),
               color=alt.Color('primary_instructor:N', title='Professor'),
               tooltip=[
                   alt.Tooltip('course_code:N', title='Course'),
                   alt.Tooltip('primary_instructor:N', title='Professor'),
                   alt.Tooltip('weighted_gpa:Q', title='Weighted GPA', format='.2f')
               ]
           )
           .properties(
               width=chart_width,
               height=chart_height,
               title='Weighted GPA by Course and Professor'
           )
           .configure_title(fontSize=18, anchor='middle')
    )

In [293]:
make_grouped_bar_chart(df)

alt.Chart(...)